# Loading Feature Information

Regardless of whether a `ChromatogramLoader` or `SpectrumLoader` is used, *Massseer* can read information from the `OpenSwath` or `DIA-NN` output on the features found for a particular peptide. 

As a demonstration, we will compare the features found for the peptide `AFVDFLSDEIK` with a charge state of 2

In [1]:
# Please run this before executing any cell
import os
os.chdir("../../tests/test_data/") #### Insert path to data, this is the path to the tutorial data. 

## Loading Transition Group Features

For this example, lets initialize an `MzMLLoader` with both DIA-NN outputs and `OpenSWATH` outputs so that we can compare their features. 

In [2]:
from massseer.loaders import MzMLDataLoader
loader_osw = MzMLDataLoader(dataFiles="ionMobilityTest2/ionMobilityTest2.mzML",
                        rsltsFile="ionMobilityTest2/ionMobilityTest2.osw",
                        rsltsFileType="OpenSWATH")

loader_diann = MzMLDataLoader(dataFiles="ionMobilityTest2/ionMobilityTest2.mzML",
                        rsltsFile="ionMobilityTest2/ionMobilityTest2-diannReport.tsv",
                        libraryFile="ionMobilityTest2/ionMobilityTest2Library.tsv",
                        rsltsFileType='DIA-NN')

[2024-01-02 11:48:03,243] MzMLDataAccess - INFO - Opening ionMobilityTest2/ionMobilityTest2.mzML file...: Elapsed 0.09108734130859375 ms
[2024-01-02 11:48:03,244] MzMLDataAccess - INFO - There are 125 spectra and 0 chromatograms.
[2024-01-02 11:48:03,245] MzMLDataAccess - INFO - There are 25 MS1 spectra and 100 MS2 spectra.
[2024-01-02 11:48:03,297] MzMLDataAccess - INFO - Opening ionMobilityTest2/ionMobilityTest2.mzML file...: Elapsed 0.044258832931518555 ms
[2024-01-02 11:48:03,298] MzMLDataAccess - INFO - There are 125 spectra and 0 chromatograms.
[2024-01-02 11:48:03,299] MzMLDataAccess - INFO - There are 25 MS1 spectra and 100 MS2 spectra.


*Massseer* uses `TransitionGroupFeature` information to store metadata on the feature detected. At minimum this contains the retention time boundaries detected by the software tool but other information as described below is also found. 

TODO: Other columns and their descriptions.

To load the top  `TransitionGroupFeature` we can use the `loadTopTransitionGroupFeature()` method. 

In [3]:
features_osw = loader_osw.loadTopTransitionGroupFeature("AFVDFLSDEIK", 2)
features_osw

{'ionMobilityTest2/ionMobilityTest2.mzML': -------- TransitionGroupFeature --------
 leftBoundary: 6235.8486328125
 rightBoundary: 6248.42822265625
 areaIntensity: 332556.620606927
 consensusApex: 6242.15
 consensusApexIntensity: 332556.620606927
 qvalue: 3.5084067486223456e-05
 consensusApexIM: -1
 precursor_mz: None
 precursor_charge: 2
 product_annotations: None
 product_mz: None
 sequence: AFVDFLSDEIK}

In [4]:
features_diann = loader_diann.loadTopTransitionGroupFeature("AFVDFLSDEIK", 2)
features_diann

{'ionMobilityTest2/ionMobilityTest2.mzML': -------- TransitionGroupFeature --------
 leftBoundary: 6236.052702
 rightBoundary: 6248.63205
 areaIntensity: 1137201.5
 consensusApex: 6241.44882
 consensusApexIntensity: None
 qvalue: 7.964108227e-05
 consensusApexIM: 0.9800000191
 precursor_mz: None
 precursor_charge: 2
 product_annotations: None
 product_mz: None
 sequence: AFVDFLSDEIK}

This method returns a dictionary where the keys are the runnames and the value is a list of TransitionGroupFeatures. 

Here, we can see that both `OpenSwath` and `DIA-NN` are detecting the same feature since the left and right boundaries and consensusApex are approximately equal. The intensities are different due to the different strategies that `OpenSWATH` and `DIA-NN` use to compute intensity. `OpenSWATH` sums up the intensity across all fragments while `DIA-NN` sums up the intensity across the top 3 fragment ions.  

### Loading The Top Transition Group Features as a Pandas DataFrame

For easier data manipulation, *Massseer* can also load feature information directly into a pandas dataframe using the `loadTransitionGroupFeaturesDf` method. However, this limits the usage with downstream *masseer* tools.

In [5]:
loader_osw.loadTopTransitionGroupFeatureDf("AFVDFLSDEIK", 2)

,filename,feature_id,precursor_id,precursor_mz,precursor_charge,areaIntensity,consensusApexIntensity,sequence,consensusApex,leftBoundary,rightBoundary,consensusApexIM,ms2_dscore,peakgroup_rank,qvalue,run_id
0,ionMobilityTest2/ionMobilityTest2.mzML,-4054571632749145316,33018,642.3295,2,2848190.0,332556.620607,AFVDFLSDEIK,6242.15,6235.848633,6248.428223,-1,5.932165,1.0,0.000035,2870707016753918864


In [6]:
loader_diann.loadTopTransitionGroupFeatureDf("AFVDFLSDEIK", 2)

,filename,Run,leftBoundary,rightBoundary,areaIntensity,qvalue,consensusApex,consensusApexIntensity,precursor_charge,sequence,consensusApexIM
0,ionMobilityTest2/ionMobilityTest2.mzML,ionMobilityTest2,6236.052702,6248.63205,1137201.5,0.00008,6241.44882,NaN,2,AFVDFLSDEIK,0.98


## Loading All TransitionGroupFeatures

By default, `OpenSWATH` computes several features for a target peptide and the top feature is determined by `PyProphet` to load all of the features identified by `OpenSwath` for a particular precursor, we can use the `loadTransitionGroupFeatures` method. 

In [7]:
loader_osw.loadTransitionGroupFeatures("AFVDFLSDEIK", 2)

{'ionMobilityTest2/ionMobilityTest2.mzML': [-------- TransitionGroupFeature --------
  leftBoundary: 6235.8486328125
  rightBoundary: 6248.42822265625
  areaIntensity: 2848190.0
  consensusApex: 6242.15
  consensusApexIntensity: 332556.620606927
  qvalue: 3.5084067486223456e-05
  consensusApexIM: -1
  precursor_mz: None
  precursor_charge: 2
  product_annotations: None
  product_mz: None
  sequence: AFVDFLSDEIK,
  -------- TransitionGroupFeature --------
  leftBoundary: 6255.64599609375
  rightBoundary: 6266.51513671875
  areaIntensity: 35433.9
  consensusApex: 6256.67
  consensusApexIntensity: 5550.48670984957
  qvalue: 0.0001776217262565
  consensusApexIM: -1
  precursor_mz: None
  precursor_charge: 2
  product_annotations: None
  product_mz: None
  sequence: AFVDFLSDEIK]}

This method returns a dictionary where the keys are the run names and the values are a list of all the TransitionGroupFeatures found. Here we can see that `OpenSWATH` finds an additional feature however in general this should be ignore because the top `TransitionGruopFeature` has a lower `q-value` 

<div class="alert alert-info">

**Note:**

DIA-NN only outputs one feature per precursor so calling the `loadTransitionGroupFeatures()` method will output esentially the same (only difference is that `loadTransitionGroupFeatures()` outputs a list of `TransitionGroupFeatures`)   

</div>

In [8]:
loader_diann.loadTransitionGroupFeatures("AFVDFLSDEIK", 2)

{'ionMobilityTest2/ionMobilityTest2.mzML': [-------- TransitionGroupFeature --------
  leftBoundary: 6236.052702
  rightBoundary: 6248.63205
  areaIntensity: 1137201.5
  consensusApex: 6241.44882
  consensusApexIntensity: None
  qvalue: 7.964108227e-05
  consensusApexIM: 0.9800000191
  precursor_mz: None
  precursor_charge: 2
  product_annotations: None
  product_mz: None
  sequence: AFVDFLSDEIK]}

## Loading All TransitionGroupFeatures In a Pandas DataFrame

To load all TransitionGroupFeatures in a pandas dataframe, the `loadTransitionGroupFeaturesDf()` method can be used. 

In [9]:
features_df_osw = loader_osw.loadTransitionGroupFeaturesDf("AFVDFLSDEIK", 2)
features_df_osw

,filename,feature_id,precursor_id,precursor_mz,precursor_charge,areaIntensity,consensusApexIntensity,sequence,consensusApex,leftBoundary,rightBoundary,consensusApexIM,ms2_dscore,peakgroup_rank,qvalue,run_id
0,ionMobilityTest2/ionMobilityTest2.mzML,-4054571632749145316,33018,642.3295,2,2848190.0,332556.620607,AFVDFLSDEIK,6242.15,6235.848633,6248.428223,-1,5.932165,1.0,0.000035,2870707016753918864
1,ionMobilityTest2/ionMobilityTest2.mzML,8869432762985074175,33018,642.3295,2,35433.9,5550.486710,AFVDFLSDEIK,6256.67,6255.645996,6266.515137,-1,5.131462,2.0,0.000178,2870707016753918864


Althuogh the Pandas DataFrame output is incompatible with further *Masseer* analysis, the pandas dataframe allows for greater flexibity with alternative analysis. For example, we can calculate the peakWidth of all features as shown below.

In [10]:
features_df_osw['peakWidth'] = features_df_osw['rightBoundary'] - features_df_osw['leftBoundary']
features_df_osw

,filename,feature_id,precursor_id,precursor_mz,precursor_charge,areaIntensity,consensusApexIntensity,sequence,consensusApex,leftBoundary,rightBoundary,consensusApexIM,ms2_dscore,peakgroup_rank,qvalue,run_id,peakWidth
0,ionMobilityTest2/ionMobilityTest2.mzML,-4054571632749145316,33018,642.3295,2,2848190.0,332556.620607,AFVDFLSDEIK,6242.15,6235.848633,6248.428223,-1,5.932165,1.0,0.000035,2870707016753918864,12.579590
1,ionMobilityTest2/ionMobilityTest2.mzML,8869432762985074175,33018,642.3295,2,35433.9,5550.486710,AFVDFLSDEIK,6256.67,6255.645996,6266.515137,-1,5.131462,2.0,0.000178,2870707016753918864,10.869141


We can also just load the top features in a dataframe if desired.

Proof Intensities are the same

In [11]:
import pandas as pd
df = pd.read_csv("ionMobilityTest2/ionMobilityTest2-diannReport.tsv", sep='\t')
sum([ float(i) for i in df['Fragment.Quant.Raw'].iloc[1].split(';')[:-1] ])

3079256.37492